#DataFrame Basics

#important links

https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=functions


In [1]:
!pip install pyspark

     |████████████████████████████████| 217.8MB 66kB/s 
     |████████████████████████████████| 204kB 58.3MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=d8a1f742cf0c6b092dfdd3745a8f8e0f5472352c75e55ac414334cd1c01cfc07
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [2]:
!sudo apt install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jre x11-utils
0 upgraded, 13 newly installed, 0 to remove and 25 not upgraded.
Need to get 6,970 kB of archives.
After this operation, 20.0 MB of additional disk space will be used.
Get

In [3]:
!sudo update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [4]:
! java -version

openjdk version "1.8.0_242"
OpenJDK Runtime Environment (build 1.8.0_242-8u242-b08-0ubuntu3~18.04-b08)
OpenJDK 64-Bit Server VM (build 25.242-b08, mixed mode)


In [5]:
!pip install findspark

In [0]:
import findspark
import os
#spark_location='/opt/spark-2.4.3/' # Set your own
java8_location= '/usr/lib/jvm/java-8-openjdk-amd64' # Set your own
os.environ['JAVA_HOME'] = java8_location
#findspark.init(spark_home=spark_location) 

In [0]:
import pyspark
import numpy as np
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [0]:
from pyspark.sql import SparkSession

#make a session for pyspark
spark = SparkSession.builder.appName("Python Spark Sql Basic Example").config("spark.some.config.option", "some_value").getOrCreate()

In [64]:
spark

In [0]:
#read a file 
df = spark.read.csv('/content/sample_data/california_housing_train.csv', header='true', inferSchema = True) #inferSchema automatically loads the columns in the variable form in which they were saved

In [170]:
#print schema to see the variable types
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [171]:
df.collect()[:5] #.collect() returns a RDD in form of list with each element as row object   

[Row(longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0),
 Row(longitude=-114.56, latitude=33.69, housing_median_age=17.0, total_rooms=720.0, total_bedrooms=174.0, population=333.0, households=117.0, median_income=1.6509, median_house_value=85700.0),
 Row(longitude=-114.57, latitude=33.64, housing_median_age=14.0, total_rooms=1501.0, total_bedrooms=337.0, population=515.0, households=226.0, median_income=3.1917, median_house_value=73400.0),
 Row(longitude=-114.57, latitude=33.57, housing_median_age=20.0, total_rooms=1454.0, total_bedrooms=326.0, population=624.0, households=262.0, median_income=1.925, median_house_value=65500.0)]

In [217]:
#get shape of df
df.count(),len(df.columns)

(17000, 12)

In [0]:
# by default pyspark reads all the column  as string--- to type cast use select Expr
df = df.selectExpr("cast(longitude as float)  as lon", 
                   "cast(housing_median_age as int) as hm_age",
                   "*")

In [173]:
df.printSchema()

root
 |-- lon: float (nullable = true)
 |-- hm_age: integer (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [174]:
#whenever we do .show() pyspark starts the computation---- Spark DF does lazy computation
df.show(10)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31|    15|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|-114.47|    19|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|-114.56|    17|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|-114.57|    14|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|-114.57|    20|  -114.57|   33.57|      

In [175]:
#select top n rowes
df.head(5)

[Row(lon=-114.30999755859375, hm_age=15, longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(lon=-114.47000122070312, hm_age=19, longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0),
 Row(lon=-114.55999755859375, hm_age=17, longitude=-114.56, latitude=33.69, housing_median_age=17.0, total_rooms=720.0, total_bedrooms=174.0, population=333.0, households=117.0, median_income=1.6509, median_house_value=85700.0),
 Row(lon=-114.56999969482422, hm_age=14, longitude=-114.57, latitude=33.64, housing_median_age=14.0, total_rooms=1501.0, total_bedrooms=337.0, population=515.0, households=226.0, median_income=3.1917, median_house_value=73400.0),
 Row(lon=-114.56999969482422, hm_age=20, longitude=-114.57, latitude=33.57, hous

In [0]:
#by default pyspark does not maintain indexing
# we use window function in spark df just as in sql
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
w = Window().partitionBy("hm_age").orderBy("hm_age")
df1 = df.withColumn("row_num", row_number().over(w))

In [177]:
df1.show(2)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|row_num|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-------+
|-114.61|    31|  -114.61|   34.83|              31.0|     2478.0|         464.0|    1346.0|     479.0|        3.212|           70400.0|      1|
|-115.57|    31|  -115.57|   32.83|              31.0|     1494.0|         289.0|     959.0|     284.0|       3.5282|           67500.0|      2|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-------+
only showing top 2 rows



In [178]:
#run a operation on pyspark df through sql query
df.createOrReplaceTempView("people") # convert df into sql_table
sqlDF = spark.sql("SELECT * FROM people where hm_age = 15")
sqlDF.show(2)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31|    15|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|-114.63|    15|  -114.63|   32.76|              15.0|     1448.0|         378.0|     949.0|     300.0|       0.8585|           45000.0|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 2 rows



In [179]:
#select a column from dataframe
df.select("longitude").show(5)

+---------+
|longitude|
+---------+
|  -114.31|
|  -114.47|
|  -114.56|
|  -114.57|
|  -114.57|
+---------+
only showing top 5 rows



In [180]:
#Describe the DataFrame
df.describe().show()

+-------+-------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|                lon|            hm_age|          longitude|          latitude|housing_median_age|      total_rooms|   total_bedrooms|        population|       households|     median_income|median_house_value|
+-------+-------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              17000|             17000|              17000|             17000|             17000|            17000|            17000|             17000|            17000|             17000|             17000|
|   mean|-119.56210822834689| 28.58935294117647|-119.56210823529375|  35.6252247058827| 28.58935294117647|2643.664411764706|539.

In [181]:
#dealing with nan values (drop nan)
df.dropna().show(5)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31|    15|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|-114.47|    19|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|-114.56|    17|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|-114.57|    14|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|-114.57|    20|  -114.57|   33.57|      

In [182]:
#dealing with nan values (substitute with -1)
df.fillna(-1).show(5)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31|    15|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|-114.47|    19|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|-114.56|    17|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|-114.57|    14|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|-114.57|    20|  -114.57|   33.57|      

In [183]:
# get distinct values from a column (substitute of unique in pandas)
np.array(df[['total_bedrooms']].distinct().collect(), dtype=np.float)

array([[ 496.],
       [ 692.],
       [ 596.],
       ...,
       [ 110.],
       [ 947.],
       [1022.]])

In [184]:
# get distinct values count from a column (substitute of nunique in pandas)
df[['total_bedrooms']].distinct().count()

1848

In [185]:
#get all the columns
df.columns

['lon',
 'hm_age',
 'longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value']

In [186]:
#access all the values of a column 
list(zip(*df.select("longitude").collect()))[0]

(-114.31,
 -114.47,
 -114.56,
 -114.57,
 -114.57,
 -114.58,
 -114.58,
 -114.59,
 -114.59,
 -114.6,
 -114.6,
 -114.6,
 -114.61,
 -114.61,
 -114.63,
 -114.65,
 -114.65,
 -114.65,
 -114.66,
 -114.67,
 -114.68,
 -114.73,
 -114.94,
 -114.98,
 -115.22,
 -115.32,
 -115.37,
 -115.37,
 -115.37,
 -115.37,
 -115.38,
 -115.38,
 -115.39,
 -115.4,
 -115.4,
 -115.41,
 -115.46,
 -115.48,
 -115.48,
 -115.49,
 -115.49,
 -115.49,
 -115.49,
 -115.5,
 -115.5,
 -115.5,
 -115.51,
 -115.51,
 -115.51,
 -115.51,
 -115.52,
 -115.52,
 -115.52,
 -115.52,
 -115.52,
 -115.52,
 -115.52,
 -115.52,
 -115.52,
 -115.53,
 -115.53,
 -115.53,
 -115.53,
 -115.53,
 -115.54,
 -115.54,
 -115.54,
 -115.54,
 -115.54,
 -115.55,
 -115.55,
 -115.55,
 -115.55,
 -115.55,
 -115.55,
 -115.55,
 -115.56,
 -115.56,
 -115.56,
 -115.56,
 -115.56,
 -115.56,
 -115.56,
 -115.56,
 -115.56,
 -115.56,
 -115.57,
 -115.57,
 -115.57,
 -115.57,
 -115.57,
 -115.57,
 -115.57,
 -115.57,
 -115.58,
 -115.58,
 -115.58,
 -115.58,
 -115.58,
 -115.59,
 -115.59

In [187]:
#access a particular values from df 
list(zip(*df.collect()))[1][2] #column index,row index

17

In [188]:
df.dtypes

[('lon', 'float'),
 ('hm_age', 'int'),
 ('longitude', 'double'),
 ('latitude', 'double'),
 ('housing_median_age', 'double'),
 ('total_rooms', 'double'),
 ('total_bedrooms', 'double'),
 ('population', 'double'),
 ('households', 'double'),
 ('median_income', 'double'),
 ('median_house_value', 'double')]

In [189]:
df.take(5)

[Row(lon=-114.30999755859375, hm_age=15, longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(lon=-114.47000122070312, hm_age=19, longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0),
 Row(lon=-114.55999755859375, hm_age=17, longitude=-114.56, latitude=33.69, housing_median_age=17.0, total_rooms=720.0, total_bedrooms=174.0, population=333.0, households=117.0, median_income=1.6509, median_house_value=85700.0),
 Row(lon=-114.56999969482422, hm_age=14, longitude=-114.57, latitude=33.64, housing_median_age=14.0, total_rooms=1501.0, total_bedrooms=337.0, population=515.0, households=226.0, median_income=3.1917, median_house_value=73400.0),
 Row(lon=-114.56999969482422, hm_age=20, longitude=-114.57, latitude=33.57, hous

In [190]:
df.groupBy('total_rooms').max().show(5)

+-----------+--------+-----------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|total_rooms|max(lon)|max(hm_age)|max(longitude)|max(latitude)|max(housing_median_age)|max(total_rooms)|max(total_bedrooms)|max(population)|max(households)|max(median_income)|max(median_house_value)|
+-----------+--------+-----------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|      934.0| -116.26|         45|       -116.26|        37.66|                   45.0|           934.0|              284.0|          893.0|          255.0|            5.3209|               185000.0|
|     3980.0| -117.11|         16|       -117.11|         37.7|                   16.0|          3980.0|              736.0|         3174.0|          679.0|            5.7068|               256700.0|


In [191]:
df.groupBy('total_rooms').min().show(5)

+-----------+--------+-----------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|total_rooms|min(lon)|min(hm_age)|min(longitude)|min(latitude)|min(housing_median_age)|min(total_rooms)|min(total_bedrooms)|min(population)|min(households)|min(median_income)|min(median_house_value)|
+-----------+--------+-----------+--------------+-------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|      934.0| -120.98|         10|       -120.98|        33.07|                   10.0|           934.0|              148.0|          401.0|          162.0|            0.9336|                83700.0|
|     3980.0| -121.76|          9|       -121.76|        32.82|                    9.0|          3980.0|              682.0|         1705.0|          647.0|            2.6607|               175000.0|


In [192]:
df.groupBy('total_rooms').count().show(5)

+-----------+-----+
|total_rooms|count|
+-----------+-----+
|      934.0|    5|
|     3980.0|    2|
|     4142.0|    1|
|      596.0|    1|
|     1761.0|    5|
+-----------+-----+
only showing top 5 rows



In [193]:
df.groupBy('total_rooms').avg().show(5)

+-----------+-------------------+-----------+--------------+------------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|total_rooms|           avg(lon)|avg(hm_age)|avg(longitude)|     avg(latitude)|avg(housing_median_age)|avg(total_rooms)|avg(total_bedrooms)|avg(population)|avg(households)|avg(median_income)|avg(median_house_value)|
+-----------+-------------------+-----------+--------------+------------------+-----------------------+----------------+-------------------+---------------+---------------+------------------+-----------------------+
|      934.0|-118.46800079345704|       27.0|      -118.468|            34.684|                   27.0|           934.0|              230.2|          570.4|          204.6|            2.8794|               143480.0|
|     3980.0|-119.43500137329102|       12.5|      -119.435|35.260000000000005|                   12.5|          3980.0|              70

In [194]:
# to apply multiple aggregate functions with groupby
grouped_data = df.groupBy('total_rooms')
grouped_data.agg({'hm_age':'max', 'lon':'count'}).show(5)

+-----------+-----------+----------+
|total_rooms|max(hm_age)|count(lon)|
+-----------+-----------+----------+
|      934.0|         45|         5|
|     3980.0|         16|         2|
|     4142.0|         37|         1|
|      596.0|         25|         1|
|     1761.0|         42|         5|
+-----------+-----------+----------+
only showing top 5 rows



In [195]:
#order by ascending
df.orderBy('total_rooms').show(5)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-117.79|     4|  -117.79|   35.21|               4.0|        2.0|           2.0|       6.0|       2.0|        2.375|          137500.0|
| -122.5|    52|   -122.5|   37.79|              52.0|        8.0|           1.0|      13.0|       1.0|      15.0001|          500001.0|
|-118.44|    46|  -118.44|   34.28|              46.0|       11.0|          11.0|      24.0|      13.0|        2.875|          162500.0|
|-122.29|    46|  -122.29|   37.81|              46.0|       12.0|           4.0|      18.0|       7.0|       0.4999|           67500.0|
|-118.04|    36|  -118.04|    33.9|      

In [196]:
#orderby descending
df.orderBy(df['total_rooms'].desc()).show(5)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-117.74|     4|  -117.74|   33.89|               4.0|    37937.0|        5471.0|   16122.0|    5189.0|       7.4947|          366300.0|
|-121.79|    11|  -121.79|   36.64|              11.0|    32627.0|        6445.0|   28566.0|    6082.0|       2.3087|          118800.0|
|-117.78|     8|  -117.78|   34.03|               8.0|    32054.0|        5290.0|   15507.0|    5050.0|       6.0191|          253900.0|
|-118.78|     9|  -118.78|   34.16|               9.0|    30405.0|        4093.0|   12873.0|    3931.0|       8.0137|          399200.0|
|-117.12|     4|  -117.12|   33.52|      

In [197]:
# perform joins on dataframes
#self join
new_df = df.join(df, on=['total_bedrooms', 'hm_age', 'total_rooms'], how='inner')
new_df.show(5)

+--------------+------+-----------+-------+---------+--------+------------------+----------+----------+-------------+------------------+-------+---------+--------+------------------+----------+----------+-------------+------------------+
|total_bedrooms|hm_age|total_rooms|    lon|longitude|latitude|housing_median_age|population|households|median_income|median_house_value|    lon|longitude|latitude|housing_median_age|population|households|median_income|median_house_value|
+--------------+------+-----------+-------+---------+--------+------------------+----------+----------+-------------+------------------+-------+---------+--------+------------------+----------+----------+-------------+------------------+
|        1283.0|    15|     5612.0|-114.31|  -114.31|   34.19|              15.0|    1015.0|     472.0|       1.4936|           66900.0|-114.31|  -114.31|   34.19|              15.0|    1015.0|     472.0|       1.4936|           66900.0|
|        1901.0|    19|     7650.0|-114.47|  -11

In [198]:
#concat two dataframes
new_df = df.union(df)
new_df.show()

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31|    15|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|-114.47|    19|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|-114.56|    17|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|-114.57|    14|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|-114.57|    20|  -114.57|   33.57|      

In [199]:
# add a column using a user defined function
def add_labels(indx):
    return indx + 200  
labels_udf = F.udf(add_labels, T.FloatType())
new_df = df.withColumn('dummy', labels_udf('lon'))
new_df.show(2)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|dummy|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----+
|-114.31|    15|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|85.69|
|-114.47|    19|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|85.53|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----+
only showing top 2 rows



In [200]:
df.printSchema()

root
 |-- lon: float (nullable = true)
 |-- hm_age: integer (nullable = true)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [0]:
#Using pyspark groupby apply function with pandas UDF
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)
def myFunc(pdf):
  #do anything here just as we do in pandas dataframe
  pdf['population'] = 3
  return pdf

df1 = df.groupby('total_rooms').apply(myFunc)

In [202]:
df1.show(5)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-117.94|    40|  -117.94|   33.78|              40.0|      299.0|          68.0|       3.0|      70.0|       3.0125|          166100.0|
|-118.17|    37|  -118.17|    34.1|              37.0|      299.0|          89.0|       3.0|      92.0|       1.3125|          145800.0|
|-118.26|    49|  -118.26|   34.03|              49.0|      299.0|          90.0|       3.0|      68.0|       1.2096|          100000.0|
|-117.17|    25|  -117.17|   33.18|              25.0|      596.0|         115.0|       3.0|     137.0|       3.0221|          214300.0|
|-121.32|    25|  -121.32|   38.57|      

In [0]:
import geopy.distance  as gp

In [0]:
from pyspark.sql.functions import monotonically_increasing_id 
df = df.withColumn("id", monotonically_increasing_id())

In [0]:
#Using pyspark groupby apply with many pandas like functions inside pandas UDF
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf(df.schema, PandasUDFType.GROUPED_MAP)

def myFunc(pdf):
  
  pdf['population'] = [gp.vincenty((pdf["latitude"].iloc[i], pdf["longitude"].iloc[i]), (33.78,-117.94)).km for i in range(pdf.shape[0])]
  
  return pdf

df2 = df.groupby('id').apply(myFunc)

In [206]:
df2.show(5)

+-------+------+---------+--------+------------------+-----------+--------------+------------------+----------+-------------+------------------+----+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|        population|households|median_income|median_house_value|  id|
+-------+------+---------+--------+------------------+-----------+--------------+------------------+----------+-------------+------------------+----+
|-115.37|    30|  -115.37|   32.82|              30.0|     1602.0|         322.0|261.96027269469243|     335.0|       3.5735|           71100.0|  26|
|-115.37|    23|  -115.37|   32.81|              23.0|     1458.0|         294.0| 262.4253200688513|     275.0|       2.3594|           74300.0|  29|
|-116.98|    12|  -116.98|   32.88|              12.0|     7320.0|        1279.0| 133.9857390730649|    1249.0|       4.3952|          151700.0| 474|
|-117.09|    26|  -117.09|   32.73|              26.0|     3114.0|         686.0|  140.836028805934|

In [168]:
#applying a function on pandas dataframe without groupby and with pandas udf
#But SCALAR_ITER is available in Databricks Runtime 5.5 and above

from pyspark.sql.functions import pandas_udf, PandasUDFType

# Input/output are both a pandas.Series of doubles
@pandas_udf('double', PandasUDFType.SCALAR_ITER)
def pandas_plus_one(df_itr):
    for x in df_itr:
      yield geopy.distance.distance((x["latitude"], x["longitude"]), (33.78,-117.94)).km

df3 = df.withColumn('v2', pandas_plus_one(df))
df3.show()

AttributeError: ignored

In [0]:
#using isin function in pyspark df
from pyspark.sql import functions as F
 
lst = [1602,3114]

df4 = df.filter(F.col('total_rooms').isin(lst))

In [210]:
df4.show(10)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|   id|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----+
|-115.37|    30|  -115.37|   32.82|              30.0|     1602.0|         322.0|    1130.0|     335.0|       3.5735|           71100.0|   26|
|-116.21|    19|  -116.21|   33.71|              19.0|     3114.0|         787.0|    3157.0|     772.0|       1.7083|           82200.0|  154|
|-117.09|    26|  -117.09|   32.73|              26.0|     3114.0|         686.0|    1948.0|     660.0|       2.8942|          124100.0|  964|
|-117.21|    14|  -117.21|   32.89|              14.0|     3114.0|         773.0|    1592.0|     776.0|       3.3176|          156100.0| 1531|

In [0]:
#Applying multiple conditions just as in pandas
df5 = df.withColumn("room_profile", F.when(df.total_rooms >= 1000, "Big").otherwise(F.when(df.total_rooms >= 500, "Medium").otherwise("Small")))

In [213]:
df5.show(5)

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---+------------+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value| id|room_profile|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---+------------+
|-114.31|    15|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|  0|         Big|
|-114.47|    19|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|  1|         Big|
|-114.56|    17|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|  2|      Medium|
|-114.57|    14|  -114.57|   33.64|              14.0|     1501.0|         3

In [214]:
#filter based on conditions 
df.filter(df["total_rooms"]==720).show()

+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----+
|    lon|hm_age|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|   id|
+-------+------+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+-----+
|-114.56|    17|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|    2|
|-117.88|    29|  -117.88|   33.74|              29.0|      720.0|         174.0|    1045.0|     181.0|       3.1964|          151900.0| 3385|
|-118.31|    28|  -118.31|   34.09|              28.0|      720.0|         267.0|     891.0|     265.0|       1.8977|          100000.0| 6930|
|-121.54|    27|  -121.54|   39.33|              27.0|      720.0|         150.0|     359.0|     138.0|       2.5313|           61300.0|12367|

In [0]:
#convet spark df to pandas df
pandas_df = df.toPandas()

In [0]:
#convert pandas df to spark df
spark_df = spark.createDataFrame(pandas_df)